In [1]:
import os
import csv
import pandas as pd

In [2]:
games_file = os.pardir + '/output_data/games_with_sba.csv'
input_directory = os.pardir + '/input_data/2018pbp'
output_directory = os.pardir + '/output_data/play_by_play'

In [3]:
games_df = pd.read_csv(games_file)
num_games = len(games_df)

In [4]:
game_index = 0
current_id = games_df['game_id'][game_index]
innings_with_sba = games_df['innings'][game_index]

for file in os.listdir(input_directory):
    if(game_index >= num_games):
        break
    
    input_file = input_directory + '/' + file
    with open(input_file, 'r', encoding='utf8') as fin:
        team_pbp = fin.readlines()
       
    num_lines = len(team_pbp)
    current_line = 0
    
    while(current_line < num_lines):
        record = team_pbp[current_line].strip('\n').split(',')
        
        if(record[0] == 'id' and record[1] == current_id):
            output_file = output_directory + '/' + current_id + '.csv'
            # write this game's play-by-play data to its own file
            with open(output_file, 'w', newline='') as fout:
                writer = csv.writer(fout, delimiter=',')
                writer.writerow(record)
                current_line = current_line + 1
                while(current_line < num_lines):
                    record = team_pbp[current_line].strip('\n').split(',')
                    if(record[0] != 'id'):
                        # only write the record if it is important - important info, substitutions, or innings with stolen base attempts
                        is_important_record = False
                        if(record[0] == 'info'):
                            if(record[1] in ['visteam', 'hometeam', 'date', 'starttime']):
                                is_important_record = True
                        elif(record[0] == 'start'):
                            is_important_record = True
                        elif(record[0] == 'sub'):
                            is_important_record = True
                        elif(record[0] =='play'):
                            current_inning = record[1] + '.' + record[2]
                            if(current_inning in innings_with_sba):
                                is_important_record = True       
                        if(is_important_record):
                            writer.writerow(record)
                        current_line = current_line + 1
                    else:
                        break
                        
            game_index = game_index + 1
            if(game_index >= num_games):
                break
            current_id = games_df['game_id'][game_index]
            innings_with_sba = games_df['innings'][game_index]
                    
        else:
            current_line = current_line + 1